# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [44]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy import stats as st

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [12]:
# Your code here:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [56]:
# Your code here:
salaries.dtypes
salaries.describe()
salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [36]:
# Your code here:
nans = [i for i in salaries.columns if salaries[i].isnull().any()]

nan_count = []

for item in nans:
    nan_count.append((item,salaries[item].isnull().sum(axis = 0)))

nan_count_perc = [(a,b/len(salaries)) for a,b in nan_count] 
nan_count_perc

[('Typical Hours', 0.7582497061748485),
 ('Annual Salary', 0.24175029382515142),
 ('Hourly Rate', 0.7582497061748485)]

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [40]:
# Your code here:
hourly_salaried = [('Salary', len(salaries[salaries['Salary or Hourly'] == 'Salary'])),('Hourly', len(salaries[salaries['Salary or Hourly'] == 'Hourly']))]

hourly_salaried


[('Salary', 25161), ('Hourly', 8022)]

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [43]:
# Your code here:

departments = [salaries['Department'].unique()]
departments


[array(['POLICE', 'GENERAL SERVICES', 'WATER MGMNT', 'OEMC',
        'CITY COUNCIL', 'AVIATION', 'STREETS & SAN', 'FIRE',
        'FAMILY & SUPPORT', 'PUBLIC LIBRARY', 'TRANSPORTN',
        "MAYOR'S OFFICE", 'HEALTH', 'BUSINESS AFFAIRS', 'LAW', 'FINANCE',
        'CULTURAL AFFAIRS', 'COMMUNITY DEVELOPMENT', 'PROCUREMENT',
        'BUILDINGS', 'ANIMAL CONTRL', 'CITY CLERK', 'BOARD OF ELECTION',
        'DISABILITIES', 'HUMAN RESOURCES', 'DoIT', 'BUDGET & MGMT',
        'TREASURER', 'INSPECTOR GEN', 'HUMAN RELATIONS', 'COPA',
        'BOARD OF ETHICS', 'POLICE BOARD', 'ADMIN HEARNG',
        'LICENSE APPL COMM'], dtype=object)]

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [54]:
# Your code here:
st.ttest_1samp(salaries['Hourly Rate'][salaries['Salary or Hourly'] == 'Hourly'], 30)
#It is statitically different

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [100]:
# Your code here:
print(st.ttest_1samp(salaries['Annual Salary'][salaries['Department'] == 'POLICE'], 86000, nan_policy='omit').statistic,
st.ttest_1samp(salaries['Annual Salary'][salaries['Department'] == 'POLICE'], 86000, nan_policy='omit').pvalue/2)
#It is therefore highly unlikely that the true mean is below the last years mean 
#This means we accept the idea that salaries have been rising

3.081997005712994 0.00103017017755027


Using the `crosstab` function, find the department that has the most hourly workers. 

In [85]:
# Your code here:

pd.crosstab(salaries['Department'],salaries['Salary or Hourly']).sort_values('Hourly', ascending = False)

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [97]:
# Your code here:
salaries['Hourly Rate'][salaries['Department'] == 'STREETS & SAN'].mean()
print(st.ttest_1samp(salaries['Hourly Rate'][salaries['Department'] == 'STREETS & SAN'], 35, nan_policy = 'omit').statistic,st.ttest_1samp(salaries['Hourly Rate'][salaries['Department'] == 'STREETS & SAN'], 35, nan_policy = 'omit').pvalue/2)
#It is therefore highly unlikely that the average salary is larger that 35$
#We can therefore accept this claim

-9.567447887848152 1.6689265282355369e-21


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [114]:
# Your code here:

mu = salaries['Hourly Rate'][salaries['Salary or Hourly'] == 'Hourly'].mean()

stat = st.sem(salaries['Hourly Rate'][salaries['Salary or Hourly'] == 'Hourly'], ddof = 1)

st.t.interval(0.95, loc = mu, scale = stat, df = len(salaries['Hourly Rate'][salaries['Salary or Hourly'] == 'Hourly'])-1)


(32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [119]:
# Your code here:
mu = salaries['Annual Salary'][(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')].mean()

stat = st.sem(salaries['Annual Salary'][(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')], ddof = 1)

st.t.interval(0.95, loc = mu, scale = stat, df = len(salaries['Annual Salary'][(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')])-1)


(86177.05631531784, 86795.77269094894)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

